# Использование Pre-Trained Трансформера
----

В подавляющем большинстве случаев трансформер обучен на невообразимо большом количестве неразмеченных данных и так как у обычного разработчика нет возможности и ресурсов тренировать такие мегабольшие модели, у нас остается только один выбор - брать веса трансформера (Open-source) и дообучивать уже на нашей конкретной задаче. Поэтому рассмотрим работу и принцип работы нескольких разнообразных моделей трансформера и попробуем что-нибудь применить из них на практике. Для того, чтобы проиллюстрировать работу трансформера мною были выбраны несколько популярных архитектур:


1. General Pre-Trained Transformer - GPT (**[GPT-1](https://www.cs.ubc.ca/~amuham01/LING530/papers/radford2018improving.pdf)** , **[GPT-2](https://www.semanticscholar.org/paper/Language-Models-are-Unsupervised-Multitask-Learners-Radford-Wu/9405cc0d6169988371b2755e573cc28650d14dfe)**, **[GPT-3](https://arxiv.org/pdf/2005.14165.pdf)** - доступна только платная версия)
2. Bidirectional Encoder Representation Transformer - **BERT**
3. Bidirectional Auto-Regressive Transformer - **BART**



Важный момент, так как трансформеры способны самостоятельно строить скрытые связи в структуре данных, то мы можем использовать подход **self-supervised learning (unsupervised pre-training)**, то есть мы используем большое количество немаркированного (неразмеченного) контента (данных) для того чтобы научить модель делать правильные прогнозы.


В задаче генерации текста либо машинного перевода, подход self-supervised learning может быть представлен следующим образом:
1. Предположим что у нас имеется последовательность из n слов (элементов), этап pre-training может быть разбит на несколько частей:
2. **Первая часть:** На шаге t мы обучаем трансформер на истинной последовательности (реальный y1, ..., yi)
3. **Вторая часть:** Просим модель сделать предсказание на шаге t и сравниваем его с истинным значением yi.
4. **Третья часть:** Обновляем модель и шаг, т.е. i := i+1 и возвращаемся на шаг 1 и повторяем до тех пор пока не обработаем всю последовательность.

Вышеуказанный подход очень похож на Teacher Forcing для рекуррентных нейронных сетей.



-----

Основная идея - использовать "сырой" текст (без разметок) на этапе Pre-training, а затем будем дообучивать модельку на выполнение конкретной задачи для которой будет отложен качественно размеченный датасет меньшего размера. Это один из видов дотренировки модели.
Наример задача машинного перевода или генерации текста в данном контексте может быть представлена как однонаправленный подход (где формируем только представляения) на этапе pre-training.


### Итого полный этап тренировки Open-Sourced трансформера состоит из 2-х частей:
1. Pre-Training на большом, неразмеченном датасете (варианты feature-based approach (ELMo, ELM, Word2Vec, Glove etc) и fine-tuning approach)
2. Fine-Tuning трансформера для решения специальной задачи используя маленький размеченный датасет (свою выборку)


In [8]:
import torch
import os
import transformers # pip install transformers чтобы установить библиотеку для работы с моделями-трансформерами
import numpy as np
import sys
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Немного про GPT-подобные модели:
Тренировка GPT-1 состоит из 2 частей:
1. Pre-Training на огромном количестве неразмеченных данных (текст)
2. Fine-tuning (Supervised-fine-tuning).

Пока рассмотрим Decoder, так как Encoder плюс-минус везде и во всех архитектурах одинаков, в исследовательской работе и в большинстве источников архитектура выглядит следующим образом:

![Image](./../../src/img/gpt_one.png)

GPT подобные модели, очень хорошо показывают себя на задачах Zero-Shot (Zero-Shot Classification, Zero-Shot Captioning, Zero-Shot Generation etc). Что означает подход Zero-Shot - это такой подход который позволяет модели использоваться везде. Суть заключается в следующем - на этапе претренировки или тренировки мы обучаем модель на части данных, а на этапе инференса и тестирования модели подаем в модель невиданные ранее данные и смотрим насколько хорошо GPT справилась с задачей. Если метрики по Zero-Shot задаче показывают себя выше чем на обычной тренировке, то модель научилась контекстному распознованию данных и сформировала определенные зависимости которые позволяют самостоятельно "думать" (искать взаимосвязи) модели.

----
##### Реализации следующих версий GPT не сильно отличаются от своего предшественника, за исключением, что у модели нет нужды в дотренировке, так как добавлен механизм формирования связей, плюс сохранен контекст с ее предшественника.
----
GPT-3 добавлен механизм [Few-Shot Learning](https://arxiv.org/abs/1904.10509) для изучения контекста и понимания структуры данных. Few-Shot Learning - подход позаимствованный у человека, мы показываем пару раз нейронной сети как необходимо посутпать, а затем она пытается повторить то что увидела (то есть подаем на этапе тренировки несколько раз правильные (истинные) данные, а затем выкидываем их, чтобы на этапе инференса модель уже сама искал контекстуальную зависимость).

## Посмотрим на задачу генерации текста с использованием модели GPT-2:

In [1]:
from transformers import pipeline # https://huggingface.co/docs/transformers/main_classes/pipelines
generator = pipeline("text-generation", model="gpt2")

In [2]:
generator("Humanity today is a ", max_length=50, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Humanity today is a ills. There has been plenty of innovation, but the world we live in is still so bad, so full of unthinking, ignorant people. I still think that there is something wrong with us. The United Kingdom doesn'},
 {'generated_text': 'Humanity today is a vernacular term – a word which some interpreters attribute to a different species, the white kyllo in the Bible. To avoid confusion, a word known as quakkakuk is pronounced quakk.'},
 {'generated_text': 'Humanity today is a \ue000uice state, a state that has two \ue000um \ue000a of natural \ue000um \ue000um \ue000um in relation to Nature and to the \ue002ou'},
 {'generated_text': 'Humanity today is a vernacular of language used for everyday thought, action, perception, and action. It is a language that allows a person to think and act. It is called a person, since the person is an object of the mind'},
 {'generated_text': "Humanity today is a vernacular for what it calls social, ecological, political, a

In [4]:
# Как было сказано выше мы также можем использовать модели трансформеров для генерации новых признаков для подачи в другие модели:
import transformers
tokenizer = transformers.GPT2Tokenizer.from_pretrained("gpt2")
text = "This is another step towards true AI"
encoded_input = tokenizer(text, return_tensors='pt')
print(encoded_input)

{'input_ids': tensor([[1212,  318, 1194, 2239, 3371, 2081, 9552]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
model = transformers.GPT2Model.from_pretrained("gpt2") # https://huggingface.co/gpt2  # https://huggingface.co/docs/transformers/model_doc/gpt2
output = model(*encoded_input)
print(output["last_hidden_state"].shape)

# BERT

### BERT - Bidirectional Encoder Representation Transformer (Google, 2018, [научная статья](https://arxiv.org/abs/1810.04805))

----


~ 345 млн параметров. BERT тоже трансформер, отличие - двунаправленный механизм для расчета эмбеддингов, обработки входной последовательности, обработки скрытых состояний. Энкодинг зависит как от предшественников последовательности так и от будущих элементов. Двунаправленность не позволяет BERT заниматься задачей генерации текста (так как генерации происходит пословесно), вывод модели сразу весь батч обработанной последовательности. Подходит для задач классификации, формирования эмбеддингов данных. Главное отличие BERT Другой подход к fine-tuning модели, который состоит также из 2 этапов, но другой подход:

1. Pre-Training:
    1.1 Masked Language Model (MLM) токены последовательности случайным образом отфильтровываются при помощи так называемой маски и модель учится предсказывать убранные слова через данную маску (маска просто разряженная матрица с различными распределениями).
    1.2 Предсказание следующего предложения, но с форматом [CLS]A[SEP]B[SEP] где А и В - некие предложения, CLS токен классификации который просто указывает на начало предложения, SEP  токен конца предложения. На данном эатпе модель учится предсказывать является ли предложение В действительно продолжением предложения А.
2. Fine-Tuning

Модель BERT очень хорошо подходи для классификации последовательностей, диалоговых систем, теггирование контекста, расшифровки контекста. Широко применяется в биоинформатике (кодирование-декодирование химических соединений, ДНК, формирование связей между различными реакциями в экспериментах).

----


### Посмотрим как можно применять BERT для классификационной задачки:

In [23]:
import gzip
import os
import shutil
import time
import torch
import requests
import torchtext
import transformers
import warnings
import numpy as np
import pandas as pd
torch.backends.cudnn.deterministic = True
warnings.filterwarnings("ignore")

In [45]:
df = pd.read_csv('./data/movie_data.csv')
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [46]:
print("Observations: {}".format(len(df)))

Observations: 50000


In [48]:
train_texts = df.iloc[:35000]['review'].values
train_labels = df.iloc[:35000]['sentiment'].values

valid_texts = df.iloc[35000:40000]['review'].values
valid_labels = df.iloc[35000:40000]['sentiment'].values

test_texts = df.iloc[40000:]['review'].values
test_labels = df.iloc[40000:]['sentiment'].values

In [49]:
tokenizer = transformers.DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased') # https://huggingface.co/docs/tokenizers/python/latest/

In [50]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
valid_encodings = tokenizer(list(valid_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

In [51]:
train_encodings[0]

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [59]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [60]:
train_dataset = IMDbDataset(train_encodings, train_labels)
valid_dataset = IMDbDataset(valid_encodings, valid_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [61]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

In [55]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = transformers.DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(DEVICE)
model.train()

optim = torch.optim.Adam(model.parameters(), lr=5e-5)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

In [56]:
def compute_accuracy(model, data_loader, device):
    with torch.no_grad():
        correct_pred, num_examples = 0, 0

        for batch_idx, batch in enumerate(data_loader):

        ### Prepare data
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs['logits']
            predicted_labels = torch.argmax(logits, 1)
            num_examples += labels.size(0)
            correct_pred += (predicted_labels == labels).sum()

        return correct_pred.float()/num_examples * 100

In [ ]:
start_time = time.time()

for epoch in range(20):

    model.train()

    for batch_idx, batch in enumerate(train_loader):

        ### Prepare data
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        ### Forward
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss, logits = outputs['loss'], outputs['logits']

        ### Backward
        optim.zero_grad()
        loss.backward()
        optim.step()

        ### Logging
        if not batch_idx % 250:
            print (f'Epoch: {epoch+1:04d}/{20:04d} | '
                   f'Batch {batch_idx:04d}/{len(train_loader):04d} | '
                   f'Loss: {loss:.4f}')

    model.eval()

    with torch.set_grad_enabled(False):
        print(f'Training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nValid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')

    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')

print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

In [ ]:
del model

Fine-Tuning

In [ ]:
model = transformers.DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(DEVICE)
model.train();

In [ ]:
from transformers import Trainer, TrainingArguments


optim = torch.optim.Adam(model.parameters(), lr=5e-5)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

In [ ]:
# install dataset via pip install datasets
from datasets import load_metric
import numpy as np


metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred # logits are a numpy array, not pytorch tensor
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(
               predictions=predictions, references=labels)

In [ ]:
optim = torch.optim.Adam(model.parameters(), lr=5e-5)


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=10
)

trainer = Trainer(
    model=model,
    compute_metrics=compute_metrics,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    optimizers=(optim, None) # optimizer and learning rate scheduler
)

# force model to only use 1 GPU (even if multiple are availabe)
# to compare more fairly to previous code

trainer.args._n_gpu = 1

In [ ]:
start_time = time.time()
trainer.train()
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')

In [ ]:
trainer.evaluate()

In [ ]:
model.eval()
model.to(DEVICE)
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')